In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os 
import random

import matplotlib.font_manager as fm

from scipy import stats
from datetime import datetime
# 설치된 폰트 출력
font_list = [font.name for font in fm.fontManager.ttflist]
plt.rcParams['font.family'] = 'NanumGothic' # 한글깨짐 방지

# 논리
1. 설문조사 데이터의 인지 정도에 따라서 집단을 나눈다. 
- 2.5이상(A집단) : 어떤 활동을 통해서, 어떤 신체부위가 다치는지 잘 인지하고 있는 집단
- 2.5미만(B집단) : 어떤 활동을 통해서, 어떤 신체부위가 다치는지 잘 인지하지 못하는 집단

2. A집단과 B집단의 사고당시활동과 사고부위를 뽑는다. 
3. 사고 데이터에서 두 집단별로 각 사고당시활동별 사고부위별 사고건수를 뽑아본다. 
3. 두 집단의 사고 건수들이 이제 새로운 두 집단(A', B')
4. A'와 B'의 사고건수 집단들의 평균의 차이가 있을까? -> t-검증
- 귀무가설 : 평균의 차이가 없다 vs 대립가설 : 평균의 차이가 있다. 
    - 검증결과 : p-value 0.029... 으로 귀무가설 기각 / 대립가설 채택 => 두 집단 평균이 유의미한 차이를 보인다. 

    - 더 나아가, 차이에 대한 분석 결과, A'집단(잘 인지하는 집단)의 사고건수가 B'집단에 비해서 더 낮다. 

> 해석 : 어떤 활동을 하면서 어떤 부위를 다치는지 잘 인지하고 있는 집단이 그렇지 않은 집단보다 사고가 덜 난다. 


# Data Load

In [2]:
survey_df = pd.read_excel('./../dataset/survey_data/survey_for_Submit.xlsx', index_col=0)
experiment_df = pd.read_csv('./../dataset/after_remove_null_data/안전_기타특수제외_초중고해당학년만_나이추가.csv', index_col=0)

# 검증

1. 사고인즈 정도를 두 그룹으로 나누기
2. 경험 데이터를 사고당시활동별 사고부위별 개수 데이터로 만들기
3. 각 그룹의 사고 발생 빈도 계산
4. 두 집단 t-검증 실행

In [3]:
likert_scale = {'전혀 그렇지 않다': 5, '그렇지 않다': 4, '보통이다': 3, '그렇다': 2, '매우 그렇다': 1}
survey_df['인지'] = survey_df['인지'].map(likert_scale)

In [4]:
# 1. 사고 인지 정도를 두 그룹으로 나누기
high_perception_group = survey_df[survey_df['인지'] >= 2.5]  # 높은 인지 그룹
low_perception_group = survey_df[survey_df['인지'] < 2.5]  # 낮은 인지 그룹

In [5]:
# 2. 경험 데이터를 사고당시활동별 사고부위별 개수 데이터로 만들기
ex_group_df = experiment_df.groupby(['사고당시활동', '사고부위'])['구분'].count().reset_index()

In [6]:
# 3. 각 그룹의 사고 발생 빈도 계산
high_perception_accidents = ex_group_df[ex_group_df['사고당시활동'].isin(high_perception_group['사고당시활동']) & ex_group_df['사고부위'].isin(high_perception_group['사고부위'])]['구분']
low_perception_accidents = ex_group_df[ex_group_df['사고당시활동'].isin(low_perception_group['사고당시활동']) & ex_group_df['사고부위'].isin(low_perception_group['사고부위'])]['구분']

In [7]:
# t-검증 수행
t_stat, p_value = stats.ttest_ind(high_perception_accidents, low_perception_accidents)

print(f"높은 인지 그룹과 낮은 인지 그룹 간의 T-통계량: {t_stat}, P-값: {p_value}")

# 유의 수준 설정 (예: 0.05)
alpha = 0.05

# 결과 해석
def interpret_results(p_value, alpha=0.05):
    if p_value < alpha:
        return "귀무 가설을 기각합니다. 인지와 사고 발생 간의 평균 차이는 유의미합니다."
    else:
        return "귀무 가설을 기각하지 않습니다. 인지와 사고 발생 간의 평균 차이는 유의미하지 않습니다."

print("사고 인지와 사고 발생 간의 결과:", interpret_results(p_value, alpha))

높은 인지 그룹과 낮은 인지 그룹 간의 T-통계량: -2.6546973715934206, P-값: 0.010229293596807193
사고 인지와 사고 발생 간의 결과: 귀무 가설을 기각합니다. 인지와 사고 발생 간의 평균 차이는 유의미합니다.
